<img src="https://github.com/gantian127/soilgrids/blob/master/docs/source/_static/soilgrids_logo.png?raw=true" width='600' align='center'></a>

## Basic Info
This Jupyter Notebook was created by Tian Gan. You can test this Jupyter Notebook through [Binder](https://mybinder.org/v2/gh/gantian127/soilgrids/main?filepath=notebooks%2Fsoilgrids.ipynb) or [HydroShare](https://www.hydroshare.org/resource/7ad709d065274ad2a2a8fd08860513d5/). 

If you have any suggestion to improve the current functions of the soilgrids package, please create a github issue [here](https://github.com/gantian127/soilgrids/issues). 

Suggested citation: Gan, T. (2020). Jupyter Notebook for the soilgrids Python package, HydroShare, https://www.hydroshare.org/resource/7ad709d065274ad2a2a8fd08860513d5/

## Quick Start Tutorial 

This notebook will help you get started using the soilgrids package to download the global gridded soil information datasets. 

This tutorial includes the following sections:

1. [Brief Introduction](#section1)

   This section provides basic information about soilgrids package. 
   <br>
   
2. [Start with Examples](#section2)
   
   This section provides two examples to demonstrate how to use soilgrids to download datasets for visualization.
   <br>
   
3. [Write Your Own Code](#section3)

   This section provides guide to write your own code to download datasets for different soil properties. 
   <br>

<a id='section1'></a>
## 1. Brief Introduction

soilgrids package provides a set of functions that allows downloading of the global gridded soil information from [SoilGrids](https://www.isric.org/explore/soilgrids), a system for global digital soil mapping to map the spatial distribution of soil properties across the globe. 

soilgrids package also includes a Basic Model Interface ([BMI](https://bmi.readthedocs.io/en/latest/)), which converts the SoilGrids dataset into a reusable, plug-and-play data component for [PyMT](https://pymt.readthedocs.io/en/latest/?badge=latest) modeling framework developed by Community Surface Dynamics Modeling System ([CSDMS](https://csdms.colorado.edu/wiki/Main_Page)) 

To install soilgrids package, you can use the following command:

In [ ]:
!pip install soilgrids

<a id='section2'></a>
## 2. Start with Examples

In soilgrids package, SoilGrids class is designed for users to download datasets. BmiSoilGrids class is designed to convert the SoilGrids dataset as a data component for the [PyMT](https://pymt.readthedocs.io/en/latest/?badge=latest) modeling framework. The following examples demonstrate how to download the same dataset using SoilGrids and BmiSoilGrids for data visualization. 

### Example 1: use SoilGrids class to download data (Recommended method)

Import SoilGrids class and download data with **get_coverage_data( )** method. You can check the details of the [parameter settings](https://soilgrids.readthedocs.io/en/latest/#parameter-settings) for get_coverage_data( ) method to better understand the parameter values used in the example.

In [ ]:
import matplotlib.pyplot as plt
from soilgrids import SoilGrids

# get data from SoilGrids
soil_grids = SoilGrids()
data = soil_grids.get_coverage_data(service_id='phh2o', coverage_id='phh2o_0-5cm_mean', 
                                       west=-1784000, south=1356000, east=-1140000, north=1863000,  
                                       crs='urn:ogc:def:crs:EPSG::152160',output='test.tif')

To know the metadata of the soil dataset, you can check the "metadata" attributes. The soil dataset is stored in a GeoTiff file. The get_coverage_data( ) method returns an xarray object that reads the data from the GeoTiff file.

In [ ]:
# show metadata
for key, value in soil_grids.metadata.items():
    print('{}: {}'.format(key,value))
    
# show GeoTiff file path
print('file path: {}'.format(soil_grids.tif_file))

The soil dataset is loaded as a DataArray object (xarray). You can directly plot the data using its plot( ) method.

In [ ]:
# plot data
data.plot(figsize=(9,5))
plt.title('Mean pH between 0 and 5 cm soil depth in Senegal')

### Example 2: use BmiSoilGrids class to download data (Demonstration of how to use BMI)

Import BmiSoilGrids class and instantiate it. A configuration file (yaml file) is required to provide the parameter settings for data download. An example configure_file.yaml file is provided in the same folder with this Jupyter Notebook file. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from soilgrids import BmiSoilGrids


# initiate a data component
data_comp = BmiSoilGrids()
data_comp.initialize('config_file.yaml')

Use variable related methods from BmiSoilGrids class to check the variable information of the soil dataset. 

In [ ]:
# get variable info
var_name = data_comp.get_output_var_names()[0]
var_unit = data_comp.get_var_units(var_name)
var_location = data_comp.get_var_location(var_name)
var_type = data_comp.get_var_type(var_name)
var_grid = data_comp.get_var_grid(var_name)
print('variable_name: {} \nvar_unit: {} \nvar_location: {} \nvar_type: {} \nvar_grid: {}'.format(
    var_name, var_unit, var_location, var_type, var_grid))

Use grid related methods of BmiSoilGrids class to check the grid information of the soil dataset. 

In [ ]:
# get variable grid info 
grid_rank = data_comp.get_grid_rank(var_grid) 

grid_size = data_comp.get_grid_size(var_grid)

grid_shape = np.empty(grid_rank, int)
data_comp.get_grid_shape(var_grid, grid_shape)

grid_spacing = np.empty(grid_rank)
data_comp.get_grid_spacing(var_grid, grid_spacing)

grid_origin = np.empty(grid_rank)
data_comp.get_grid_origin(var_grid, grid_origin)

print('grid_rank: {} \ngrid_size: {} \ngrid_shape: {} \ngrid_spacing: {} \ngrid_origin: {}'.format(
    grid_rank, grid_size, grid_shape, grid_spacing, grid_origin))

Use get_value( ) method to get the data as a numpy 2D array. 

In [ ]:
# get variable data 
data = np.empty(grid_size, var_type)
data_comp.get_value(var_name, data)
data_2D = data.reshape(grid_shape)

In [ ]:
# get X, Y extent for plot
min_y, min_x = grid_origin
max_y = min_y + grid_spacing[0]*grid_shape[0]
max_x = min_x + grid_spacing[1]*grid_shape[1]
dy = grid_spacing[0]/2
dx = grid_spacing[1]/2
extent = [min_x - dx, max_x + dx, min_y - dy, max_y + dy]

# plot data
fig, ax = plt.subplots(1,1, figsize=(9,5))
im = ax.imshow(data_2D, extent=extent)
fig.colorbar(im)
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Mean pH between 0 and 5 cm soil depth in Senegal')

<a id='section3'></a>
## 3. Write Your Own Code 

In this section, you will be guided to first learn about the SoilGrids map services. Then you will write your own code to download several soil datasets. 

### Step1: Learn about SoilGrids map services###

[SoilGrids](https://www.isric.org/explore/soilgrids) system provides map services for multiple soil properties, including: pH, soil organic carbon content, bulk density, coarse fragments content, sand content, silt content, clay content, cation exchange capacity (CEC), total nitrogen as well as soil organic carbon density and soil organic carbon stock. For additional information of these maps services please visit the SoilGrids [FAQ page](https://www.isric.org/explore/soilgrids/faq-soilgrids). 

Let's first look at the detailed information of the supported map services in the SoilGrids system. The **map_services** attribute will show the map service id, url, and the variable name. 

In [ ]:
import matplotlib.pyplot as plt
from soilgrids import SoilGrids

soil_grids = SoilGrids()
soil_grids.map_services

Each map service includes multiple coverages (maps). Use **get_coverage_list( )** method to get all the coverages from a map service. In this example, it checks the available maps for "Bulk density"(bdod) map service. The coverage id "bdod_0-5cm_mean" represents a map for the mean of the bulk density between 0 to 5cm soil depth. To learn more about the meaning of the coverage id, please visit the SoilGrids [FAQ page](https://www.isric.org/explore/soilgrids/faq-soilgrids)

In [ ]:
soil_grids.get_coverage_list('bdod')

Now, we can use **get_coverage_info( )** method to check about the supported coordinate systems and the bounding box information for the "bdod_0-5cm_mean" map. This information is useful for data download.

In [ ]:
soil_grids.get_coverage_info('bdod','bdod_0-5cm_mean')

### Step2: Download soil datasets

Now we are going to download several soil datasets for the Boulder Creek area in the Colorado State. You can check your code with the answer by clicking on the "Double-click here for the solution".


Let's first define the coordinate system and the bounding box for data download. In this example, the [EPSG 4326](https://epsg.io/4326) (EPSG identifier for WGS84) coordinate system is used. So the bounding box is defined with the latitude and longitude values. Since EPSG 4326 is a geographic coordinate system, the width and height of the pixels for the subset data are required. 

In [ ]:
# define the crs and bounding box
crs = 'urn:ogc:def:crs:EPSG::4326'
west, south, east, north = [-105.38, 39.45, -104.5, 40.07]
width = 316
height = 275

Download the mean of the **silt content** between 0 and 5cm soil depth. Please note the units of the downloaded data is **g/kg**. 

In [ ]:
# download the silt content, write your code here
silt = SoilGrids()
silt_content = 

# plot the silt content data
silt_content.plot(figsize=(9,5))
plt.title('Mean silt content between 0 and 5 cm soil depth in the Boulder Creek Watershed')

*Double-click __here__ for the solution.*

<!-- Your answer is below:

silt_content = silt.get_coverage_data(service_id='silt', coverage_id='silt_0-5cm_mean', 
                                       west=west, south=south, east=east, north=north,  
                                       width=width, height=height, crs=crs, output='silt.tif')
-->

Download the mean of the **clay content** between 0 and 5cm soil depth.

In [ ]:
# download the clay content, write your code here
clay = SoilGrids()
clay_content =  

# plot data
clay_content.plot(figsize=(9,5))
plt.title('Mean clay content between 0 and 5 cm soil depth in the Boulder Creek Watershed')

*Double-click __here__ for the solution.*

<!-- Your answer is below:

clay_content = clay.get_coverage_data(service_id='clay', coverage_id='clay_0-5cm_mean', 
                                       west=west, south=south, east=east, north=north,  
                                       width=width, height=height, crs=crs, output='clay.tif')
-->

Download the mean of the **sand content** between 0 and 5cm soil depth.

In [ ]:
# download the sand content, write your code here
sand = SoilGrids()
sand_content = 

# plot data
sand_content.plot(figsize=(9,5))
plt.title('Mean sand content between 0 and 5 cm soil depth in the Boulder Creek Watershed')

*Double-click __here__ for the solution.*

<!-- Your answer is below:

sand_content = sand.get_coverage_data(service_id='sand', coverage_id='sand_0-5cm_mean', 
                                       west=west, south=south, east=east, north=north,  
                                       width=width, height=height, crs=crs, output='sand.tif')

-->